# Minimization of metabolic adjustment analysis

TODO MOMA citation

In [1]:
using COBREXA

TODO why not download+load combo?

In [2]:
download_model(
    "http://bigg.ucsd.edu/static/models/e_coli_core.json",
    "e_coli_core.json",
    "7bedec10576cfe935b19218dc881f3fb14f890a1871448fc19a9b4ee15b448d8",
)

import AbstractFBCModels.CanonicalModel as CM
import JSONFBCModels
import Clarabel

model = load_model("e_coli_core.json", CM.Model)

reference_fluxes =
    parsimonious_flux_balance_analysis(
        model,
        optimizer = Clarabel.Optimizer,
        settings = [silence],
    ).fluxes

modified_model = deepcopy(model) # avoid reference sharing

modified_model.reactions["CYTBD"]

modified_model.reactions["CYTBD"].upper_bound = 10.0

solution = metabolic_adjustment_minimization_analysis(
    modified_model,
    model;
    optimizer = Clarabel.Optimizer,
    settings = [silence],
)

solution_with_fluxes = metabolic_adjustment_minimization_analysis(
    modified_model,
    reference_fluxes;
    optimizer = Clarabel.Optimizer,
    settings = [silence],
)

solution.objective

sqrt(solution.minimal_adjustment_objective)

solution.fluxes.CYTBD

[ Info: using cached `e_coli_core.json'


9.999999999955458

TODO compare

In [3]:
optimal_solution = parsimonious_flux_balance_analysis(
    model,
    optimizer = Clarabel.Optimizer,
    settings = [silence],
)

import ConstraintTrees as C
sort(collect(C.zip(-, optimal_solution.fluxes, solution.fluxes, Float64)), by = last)

95-element Vector{Pair{Symbol, Union{Float64, ConstraintTrees.Tree{Float64}}}}:
   :EX_o2_e => -16.799492654235696
      :H2Ot => -16.579331171270734
      :CO2t => -13.584741559367512
 :EX_etoh_e => -5.5435242170094154
    :SUCOAS => -5.064375660719828
      :FRD7 => -4.073918962120799
       :PPC => -3.307339485166412
    :SUCCt3 => -3.083462145246263
 :EX_succ_e => -3.0834621450484487
       :ENO => -2.5746547772765567
            ⋮
     :ACALD => 7.46788662344968
       :FUM => 8.147837805562828
       :MDH => 8.147837806919691
  :EX_co2_e => 13.584741559366993
  :EX_h2o_e => 16.579331171269033
       :O2t => 16.79949265423621
    :NADH16 => 25.45114750288382
    :ATPS4r => 33.57037160688897
     :CYTBD => 33.59898530847451

## Faster L1-minimal adjustments

In [4]:
import GLPK

linear_solution = linear_metabolic_adjustment_minimization_analysis(
    modified_model,
    model;
    optimizer = GLPK.Optimizer,
)

linear_solution.objective

linear_solution.fluxes.CYTBD

linear_solution.linear_minimal_adjustment_objective

304.4092092993514

TODO compare again

In [5]:
sort(collect(C.zip(-, linear_solution.fluxes, solution.fluxes, Float64)), by = last)

95-element Vector{Pair{Symbol, Union{Float64, ConstraintTrees.Tree{Float64}}}}:
        :GAPD => -2.217421474277602
         :ENO => -2.1841385417330255
        :FRD7 => -2.0818742855030417
  :EX_acald_e => -1.9243624065292229
      :EX_h_e => -1.7636376825706002
         :PDH => -1.3871400070221487
      :GLCpts => -1.1700704237440451
      :SUCCt3 => -1.091417461127648
   :EX_succ_e => -1.091417460921988
      :NADH16 => -1.091417460614137
              ⋮
        :CO2t => 0.4467050816543736
        :H2Ot => 1.046555418943484
         :FUM => 1.091417460658668
         :MDH => 1.0914174620392645
 :EX_glc__D_e => 1.170070423744047
       :ACALD => 1.712501384039891
      :ACALDt => 1.9243624065292242
         :PGM => 2.184138541733022
         :PGK => 2.2174214742776126

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*